## Imports

In [1]:
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import os                                                                       
import numpy as np                                                              
import tensorflow as tf                                                         
import data.data_selector as ds                                                   
import utils.plot_functions as pf                                               
import analysis.analysis_picker as ap

In [3]:
analysis_params = {
  "model_type": "conv_lca",
  "model_name": "conv_lca_vh",#"lca_vh_ft_white",
  "data_type": "vanHateren",
  "device": "/gpu:0",
  "version": "5.0",
  "num_inference_images": 5, #How many random images to average over for inference statistics
  "clobber": True}

# Computed params
analysis_params["model_dir"] = (os.path.expanduser("~")+"/Work/Projects/"
  +analysis_params["model_name"])

In [4]:
analyzer = ap.get_analyzer(analysis_params)

In [7]:
analyzer.model_params["data_type"] = analysis_params["data_type"]
data = ds.get_data(analyzer.model_params)
analyzer.model_params["input_shape"] = [
  data["train"].num_rows*data["train"].num_cols*data["train"].num_channels]

In [23]:
#params = {'data_type': 'vanHateren', 'vectorize_data': False, 'norm_data': False, 'batch_size': 3, 'center_data': False, 'data_dir': '/home/dpaiton/Work/Datasets/vanHateren/'}
print("Analysis params:\n")
print("\n".join([key.ljust(20)+"\t"+str(analyzer.model_params[key]) for key in analyzer.model_params]))

Analysis params:

gen_plot_int        	1000
data_dir            	/home/dpaiton/Work/Datasets//vanHateren/
center_data         	True
version             	5.0
input_shape         	[65536]
norm_data           	False
norm_weights        	True
batch_size          	1
device              	/gpu:0
thresh_type         	soft
cp_load             	False
model_name          	conv_lca_vh
rectify_a           	True
patch_size_x        	16
extract_patches     	False
max_cp_to_keep      	1
num_images          	300
stride_y            	8
out_dir             	/home/dpaiton/Work/Projects/
model_type          	conv_lca
optimizer           	annealed_sgd
num_pixels          	65536
stride_x            	8
vectorize_data      	False
save_plots          	True
log_int             	10
log_to_file         	True
cp_int              	10000
num_steps           	120
image_edge_size     	256
contrast_normalize  	False
whiten_data         	True
rand_seed           	1234567890
eps                 	1e-12
patch_size_y        

In [8]:
if analysis_params["clobber"]:
  analyzer.run_analysis(data["train"].images, save_info="full_imgs")
else:
  analyzer.load_analysis(save_info="full_imgs")

ValueError: Cannot feed value of shape (300, 256, 256) for Tensor 'placeholders/input_data:0', which has shape '(?, 256, 256, 1)'

In [ ]:
stats_fig = pf.plot_stats(analyzer.run_stats,
  keys=["a_fraction_active", "recon_loss", "sparse_loss", "total_loss"],
  labels=["activity", "recon loss", "sparse loss", "total loss"])

In [ ]:
num_pixels, num_neurons = analyzer.atas.shape
atas_fig = pf.plot_data_tiled(analyzer.atas.T.reshape(num_neurons,
  int(np.sqrt(num_pixels)), int(np.sqrt(num_pixels))), normalize=False,
  title="Activity triggered averages on image data")

In [ ]:
noise_images = np.random.standard_normal(data["train"].images.shape) 
noise_evals = analyzer.evaluate_model(noise_images, analyzer.var_names)
noise_atas = analyzer.compute_atas(noise_evals["inference/activity:0"],
  noise_images)
noise_atas_fig = pf.plot_data_tiled(noise_atas.T.reshape(num_neurons,
  int(np.sqrt(num_pixels)), int(np.sqrt(num_pixels))), normalize=False,
  title="Activity triggered averages on standard normal noise data")

In [ ]:
dict_fig = pf.plot_data_tiled(analyzer.evals["weights/phi:0"].T.reshape(analyzer.model.num_neurons,
  data["train"].num_cols, data["train"].num_rows), normalize=False, title="Weights")

In [ ]:
inference_fig = pf.plot_inference_traces(analyzer.inference_stats, analyzer.model_schedule[0]["sparse_mult"])

In [ ]:
fig = pf.plot_inference_stats(analyzer.inference_stats, title="Loss During Inference")